In [117]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, query_engine, StorageContext, load_index_from_storage
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.memory import Memory
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
from core.settings import CHROMA_PATH, llm, embed_model
import chromadb,os
from llama_index.core.storage.chat_store import SimpleChatStore

In [9]:
db_path = "C:/Users/khalil/Desktop/fastapi_llamaindex/vector_stores"

In [281]:
response = llm.complete("hello")
print(response)

2025-10-12 23:35:45,404 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"


Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?


In [280]:
embeddings = embed_model.get_text_embedding("hello")
print(len(embeddings))

2025-10-12 23:06:11,702 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"


768


In [61]:
#define and ollama embedding function and will be used by chroma when creating collections.
embed_fn =OllamaEmbeddingFunction(
    url="http://192.168.2.62:11434",
    model_name="nomic-embed-text:latest",
    )

In [12]:
CHROMA_PATH

WindowsPath('C:/Users/Ds-Terminal/Desktop/rag-llamaindex-chroma/vector_stores')

In [13]:
from llama_index.core import settings
settings.llm = llm
settings.embed_model = embed_model

In [22]:
vector_client = chromadb.PersistentClient(path=CHROMA_PATH)
vector_client.__dict__

{'_identifier': 'C:\\Users\\Ds-Terminal\\Desktop\\rag-llamaindex-chroma\\vector_stores',
 'tenant': 'default_tenant',
 'database': 'default_database',
 '_server': <chromadb.api.rust.RustBindingsAPI at 0x2a5219f5550>,
 '_admin_client': <chromadb.api.client.AdminClient at 0x2a52407c690>}

In [23]:
vector_client.list_collections()

[Collection(name=string), Collection(name=stringg)]

In [24]:
vector_client.get_collection("stringg").__dict__

{'_client': <chromadb.api.rust.RustBindingsAPI at 0x2a5219f5550>,
 '_model': Collection(id=UUID('43417482-b4b8-4abc-aca9-c882f25be1cd'), name='stringg', configuration_json={'hnsw': {'space': 'cosine', 'ef_construction': 100, 'ef_search': 100, 'max_neighbors': 16, 'resize_factor': 1.2, 'sync_threshold': 1000}, 'spann': None, 'embedding_function': {'type': 'known', 'name': 'ollama', 'config': {'model_name': 'nomic-embed-text:latest', 'timeout': 60, 'url': 'http://192.168.2.62:11434'}}}, metadata=None, dimension=768, tenant='default_tenant', database='default_database', version=0, log_position=0),
 '_embedding_function': <chromadb.utils.embedding_functions.DefaultEmbeddingFunction at 0x2a5213dd850>,
 '_data_loader': None}

In [20]:

#Load documents from a folder
user_dir = "./data/stringg"
docs = SimpleDirectoryReader(user_dir).load_data()
print(f"Loaded {len(docs)} documents")

user_name = "stringg"


Loaded 1 documents


In [21]:
docs[0]

Document(id_='150835fd-0206-43f9-bbee-2fe316503054', embedding=None, metadata={'file_path': 'c:\\Users\\Ds-Terminal\\Desktop\\rag-llamaindex-chroma\\data\\stringg\\text.txt', 'file_name': 'text.txt', 'file_type': 'text/plain', 'file_size': 230, 'creation_date': '2025-10-08', 'last_modified_date': '2025-10-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='1) llm:\r\nmodel="llama3.1",\r\nbase_url="http://192.168.2.62:11434"\r\n \r\n2) embeding model: BAAI/bge-base-en\r\n \r\n3) context handling: during chat it mantains the running history of  dialogues. \r\n \r\n4) we will provide you the document ', path=None, url=None, mimetype=None), 

In [27]:
collection = vector_client.get_collection(user_name)
collection.__dict__


{'_client': <chromadb.api.rust.RustBindingsAPI at 0x2a5219f5550>,
 '_model': Collection(id=UUID('43417482-b4b8-4abc-aca9-c882f25be1cd'), name='stringg', configuration_json={'hnsw': {'space': 'cosine', 'ef_construction': 100, 'ef_search': 100, 'max_neighbors': 16, 'resize_factor': 1.2, 'sync_threshold': 1000}, 'spann': None, 'embedding_function': {'type': 'known', 'name': 'ollama', 'config': {'model_name': 'nomic-embed-text:latest', 'timeout': 60, 'url': 'http://192.168.2.62:11434'}}}, metadata=None, dimension=768, tenant='default_tenant', database='default_database', version=0, log_position=0),
 '_embedding_function': <chromadb.utils.embedding_functions.DefaultEmbeddingFunction at 0x2a5213dd850>,
 '_data_loader': None}

In [31]:
vector_store = ChromaVectorStore.from_collection(collection=collection)

In [34]:

# 6️⃣ Create storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=CHROMA_PATH)
storage_context.__dict__




Loading llama_index.core.storage.kvstore.simple_kvstore from C:\Users\Ds-Terminal\Desktop\rag-llamaindex-chroma\vector_stores\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from C:\Users\Ds-Terminal\Desktop\rag-llamaindex-chroma\vector_stores\index_store.json.


{'docstore': <llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore at 0x2a5246bd450>,
 'index_store': <llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore at 0x2a5246bd810>,
 'vector_stores': {'default': ChromaVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, collection_name=None, host=None, port=None, ssl=False, headers=None, persist_dir=None, collection_kwargs={})},
 'graph_store': <llama_index.core.graph_stores.simple.SimpleGraphStore at 0x2a5246bd950>,
 'property_graph_store': None}

In [35]:
# index = VectorStoreIndex.from_vector_store(
#         vector_store=vector_store
#     )
# print("Index loaded from existing vector store!")

In [50]:
index = load_index_from_storage(storage_context=storage_context)
chat_engine = index.as_chat_engine(llm=llm, chat_mode="condense_plus_context",system_prompt=("you are a helpful assistant that can answer general questions as well as quetions asked from the provided context. Please be concise."))

2025-10-10 00:06:00,763 - INFO - Loading all indices.


In [42]:
response = chat_engine.chat("what model names are in the context?")


2025-10-09 23:39:55,437 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"
2025-10-09 23:39:55,439 - INFO - Condensed question: What other model names are mentioned in addition to "llama3.1"?
2025-10-09 23:39:55,479 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"
2025-10-09 23:39:57,369 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"


In [55]:
print(response.response)

There are two model names mentioned in the context: "llama3.1" and "BAAI/bge-base-en".


In [ ]:
#######integrating memroy to the chat_engine ##############3

In [251]:
#create a new vector collection for the same user to store chat vectores
collection_name = user_name + "_chats"
chat_collection = vector_client.create_collection(collection_name, embedding_function=embed_fn)


In [252]:
vector_client.get_collection(collection_name).__dict__

{'_client': <chromadb.api.rust.RustBindingsAPI at 0x2a5219f5550>,
 '_model': Collection(id=UUID('40be0135-131b-4ebf-83fc-a5c619748022'), name='stringg_chats', configuration_json={'hnsw': {'space': 'cosine', 'ef_construction': 100, 'ef_search': 100, 'max_neighbors': 16, 'resize_factor': 1.2, 'sync_threshold': 1000}, 'spann': None, 'embedding_function': {'type': 'known', 'name': 'ollama', 'config': {'model_name': 'nomic-embed-text:latest', 'timeout': 60, 'url': 'http://192.168.2.62:11434'}}}, metadata=None, dimension=None, tenant='default_tenant', database='default_database', version=0, log_position=0),
 '_embedding_function': <chromadb.utils.embedding_functions.DefaultEmbeddingFunction at 0x2a5213dd850>,
 '_data_loader': None}

In [253]:
#create a vector store instance using the chat_collection
chat_vector_store = ChromaVectorStore.from_collection(chat_collection)

In [254]:
###defining static/ long term memroy
from llama_index.core.memory import StaticMemoryBlock,FactExtractionMemoryBlock,VectorMemoryBlock
memory_blocks=[
    StaticMemoryBlock(
    name="core-info",
    static_content="My name is khalil, I line in Islamabad, I work with llamaindex.",
    priority=0
),

FactExtractionMemoryBlock(
    name="extracted-info",
    llm=llm,
    max_facts=30,
    priority=1
),

VectorMemoryBlock(
    name = "vector_memory",
    vector_store=chat_vector_store,
    priority=2,
    embed_model=embed_model,
    similarity_top_k=3,
    retrieval_context_window=5,
    

),
]




In [ ]:
chat_dir = "chats"
chat_id="abc"
os.makedirs(chat_dir, exist_ok=True)

# Create user folder
user_chat_dir = os.path.join(chat_dir, "notebook_chat1")
os.makedirs(user_chat_dir, exist_ok=True)

# File path for chat
user_chat_path = os.path.join(user_chat_dir, f"{chat_id}.json")

# Create persistent SimpleChatStore
chat_store = SimpleChatStore.from_persist_path(persist_path=user_chat_path)

In [264]:
chat_store.store

{'chats': [ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='user message  chat content222...')]),
  ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')])]}

In [ ]:
from llama_index.core.llms import ChatMessage

In [220]:
chat_store.add_message(key="chats",message = ChatMessage(role="system", content="sytem some content.."))

In [169]:
chat_store.persist(persist_path=user_chat_path)


In [265]:
chat_store.store

{'chats': [ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='user message  chat content222...')]),
  ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')])]}

In [ ]:
path_manual = 'sqlite+aiosqlite:///chats/notebook_chat/abc.sqlite3'
path = "sqlite+aiosqlite:///C:/Users/Ds-Terminal/Desktop/rag-llamaindex-chroma/chats/string/abc.sqlite3"

In [ ]:
memory = Memory.from_defaults(
    session_id=user_name,
    token_limit=100,  # Normally you would set this to be closer to the LLM context window (i.e. 75,000, etc.)
    token_flush_size=20,
    chat_history_token_ratio=0.7,
    memory_blocks= memory_blocks,
    async_database_uri= path
)

In [267]:
memory.__dict__

{'token_limit': 100,
 'token_flush_size': 20,
 'chat_history_token_ratio': 0.7,
 'memory_blocks': [StaticMemoryBlock(name='core-info', description=None, priority=0, accept_short_term_memory=True, static_content=[TextBlock(block_type='text', text='My name is khalil, I line in Islamabad, I work with llamaindex.')]),
  FactExtractionMemoryBlock(name='extracted-info', description=None, priority=1, accept_short_term_memory=True, llm=Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002A5219CDA70>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x000002A51E3D0B80>, completion_to_prompt=<function default_completion_to_prompt at 0x000002A51EA4ACA0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://192.168.2.62:11434', model='llama3:latest', temperature=None, context_window=8192, request_timeout=30.0, prompt_key='prompt', json_mode=False, additional_kwargs

In [259]:
memory.get()

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='<memory>\n<core-info>\nMy name is khalil, I line in Islamabad, I work with llamaindex.\n</core-info>\n</memory>')])]

In [268]:
chat_engine_with_memory = index.as_chat_engine(llm=llm,memory = memory ,chat_mode="condense_plus_context",system_prompt=("you are a helpful assistant that can answer general questions as well as quetions asked from the provided context. Please be concise."))

In [273]:
result = await chat_engine_with_memory.achat("can YOu tell me the capital of Pakistan?")

2025-10-12 21:59:57,242 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"
2025-10-12 22:00:11,245 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"
2025-10-12 22:00:11,249 - INFO - Condensed question: What is the capital of Pakistan?
2025-10-12 22:00:13,491 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"
2025-10-12 22:00:30,780 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"
2025-10-12 22:00:33,176 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"
2025-10-12 22:00:50,566 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"


In [274]:
result.response

'According to my running history of dialogues (which includes your initial information that you live in Islamabad), I know that Islamabad is the capital of Pakistan! So, the answer is Islamabad.'

In [271]:
response.__dict__

{'text': "Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?",
 'additional_kwargs': {'tool_calls': [], 'thinking': None},
 'raw': {'model': 'llama3:latest',
  'created_at': '2025-10-13T09:25:35.003652639Z',
  'done': True,
  'done_reason': 'stop',
  'total_duration': 16188640513,
  'load_duration': 7120691371,
  'prompt_eval_count': 11,
  'prompt_eval_duration': 331030749,
  'eval_count': 26,
  'eval_duration': 8734816347,
  'message': Message(role='assistant', content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", thinking=None, images=None, tool_name=None, tool_calls=None),
  'usage': {'prompt_tokens': 11, 'completion_tokens': 26, 'total_tokens': 37}},
 'logprobs': None,
 'delta': None}

In [272]:
memory.__dict__

{'token_limit': 100,
 'token_flush_size': 20,
 'chat_history_token_ratio': 0.7,
 'memory_blocks': [StaticMemoryBlock(name='core-info', description=None, priority=0, accept_short_term_memory=True, static_content=[TextBlock(block_type='text', text='My name is khalil, I line in Islamabad, I work with llamaindex.')]),
  FactExtractionMemoryBlock(name='extracted-info', description=None, priority=1, accept_short_term_memory=True, llm=Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002A5219CDA70>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x000002A51E3D0B80>, completion_to_prompt=<function default_completion_to_prompt at 0x000002A51EA4ACA0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://192.168.2.62:11434', model='llama3:latest', temperature=None, context_window=8192, request_timeout=30.0, prompt_key='prompt', json_mode=False, additional_kwargs